

---


FSM diagram without considering multiple transitions

---



In [ ]:
import pandas as pd
from collections import defaultdict
import graphviz

# Load the dataset from the Excel file
file_path = 'SystemLogs_subset.csv'
df = pd.read_csv(file_path, header=0)

# Convert 'Date and Time' to datetime format (reference only, not used for sorting)
df['Date and Time'] = pd.to_datetime(df['Date and Time'])

# FSM state transition representation: {current_event_id: [(source, next_event_id)]}
fsm = defaultdict(list)

# Process the log file line by line in the original order of appearance
previous_event = None
previous_source = None

for _, row in df.iterrows():
    current_event = row['Event ID']
    current_source = row['Source']

    if previous_event is not None:
        # Check for self-loop and avoid duplicate transitions
        if current_event == previous_event:
            # Only add one self-loop transition for the event
            if not any(next_state == current_event for _, next_state in fsm[previous_event]):
                fsm[previous_event].append((current_source, current_event))
        else:
            # For non-self-loop transitions, add them as usual
            fsm[previous_event].append((current_source, current_event))

    previous_event = current_event
    previous_source = current_source

# Write the FSM to a text file for inspection
output_path = 'fsm_detailed_output.txt'
with open(output_path, 'w') as file:
    file.write("FSM with Transitions:\n")
    for state, transitions in fsm.items():
        file.write(f"{state} -> {[(source, next_state) for source, next_state in transitions]}\n")

print(f"The FSM output has been saved to {output_path}")

# Create an FSM diagram using Graphviz
dot = graphviz.Digraph(comment='FSM Diagram', format='png')
for state, transitions in fsm.items():
    for source, next_state in transitions:
        # Add states and edges with labels showing the source column triggering the transition
        dot.node(str(state), str(state))
        dot.node(str(next_state), str(next_state))
        dot.edge(str(state), str(next_state), label=f'{source}')

# Render the diagram to a file
diagram_path = 'fsm_diagram_original'
dot.render(diagram_path, cleanup=True)
print(f"The FSM diagram has been created and saved as {diagram_path}.png")


The FSM output has been saved to fsm_detailed_output.txt
The FSM diagram has been created and saved as fsm_diagram_original.png


<ipython-input-2-add4c9474bdd>:10: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Date and Time'] = pd.to_datetime(df['Date and Time'])




---

FSM diagram considering multiple transitions


---



In [1]:
import pandas as pd
from collections import defaultdict
import graphviz

# Load the dataset from the Excel file
file_path = 'SystemLogs_subset.csv'
df = pd.read_csv(file_path)

# Convert 'Date and Time' to datetime format (for reference only, not sorting)
df['Date and Time'] = pd.to_datetime(df['Date and Time'])

# FSM state transition representation: {current_state: [(next_state, source, level)]}
fsm = defaultdict(list)

# Read the file line by line and process transitions
previous_event = None
previous_source = None
previous_level = None  # Assuming 'Level' is the column name representing the event level

for _, row in df.iterrows():
    current_event = row['Event ID']
    current_source = row['Source']  # Assuming 'Source' is the column name in the DataFrame
    current_level = row['Level']    # Assuming 'Level' is the column name in the DataFrame

    if previous_event is not None:
        # Add the transition including level as part of the significant information
        if previous_event != current_event or previous_level != current_level:
            fsm[previous_event].append((current_event, current_source, current_level))

    previous_event = current_event
    previous_source = current_source
    previous_level = current_level

# Create a modified FSM that adds unique states for multiple transitions, excluding self-loops
fsm_modified = defaultdict(list)
suffix_count = defaultdict(int)

for state, transitions in fsm.items():
    # Remove duplicate transitions while keeping the order
    unique_transitions = []
    for transition, source, level in transitions:
        if (transition, source, level) not in unique_transitions:
            unique_transitions.append((transition, source, level))

    # Check if there are multiple unique transitions (excluding self-loops)
    if len(unique_transitions) > 1:
        for i, (transition, source, level) in enumerate(unique_transitions):
            # Create new states with suffixes for clarity
            if transition != state:  # Avoid creating suffix for self-loops
                new_state = f"{state}_{chr(97 + i)}"
                fsm_modified[state].append((new_state, source, level))
                fsm_modified[new_state].append((transition, source, level))
            else:
                fsm_modified[state].append((transition, source, level))
    else:
        fsm_modified[state] = unique_transitions

# Write the FSM to a text file
output_path = 'fsm_multiple.txt'
with open(output_path, 'w') as file:
    file.write("Original FSM:\n")
    for state, transitions in fsm.items():
        file.write(f"{state} -> {[(transition, source, level) for transition, source, level in transitions]}\n")

    file.write("\nModified FSM with New States:\n")
    for state, transitions in fsm_modified.items():
        file.write(f"{state} -> {[(transition, source, level) for transition, source, level in transitions]}\n")

print(f"The FSM output has been saved to {output_path}")

# Create a single FSM diagram using Graphviz
dot = graphviz.Digraph(comment='FSM Diagram with Levels, New States, and Sources', format='png')

# Add nodes and edges to the diagram
for state, transitions in fsm_modified.items():
    dot.node(str(state), str(state))
    for transition, source, level in transitions:
        dot.node(str(transition), str(transition))
        # Add the source and level information to the edge label
        dot.edge(str(state), str(transition), label=f"{source}, Level: {level}")

# Render the diagram to a file
diagram_path = 'fsm_diagram_multiple'
dot.render(diagram_path, cleanup=True)
print(f"The FSM diagram has been created and saved as {diagram_path}.png")


<ipython-input-1-1ef7cde0125c>:10: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Date and Time'] = pd.to_datetime(df['Date and Time'])


The FSM output has been saved to fsm_multiple.txt
The FSM diagram has been created and saved as fsm_diagram_multiple.png
